In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate


prompt = PromptTemplate.from_template("Explain about {word}.")
llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 30,
    }
)

chain = prompt | llm
chain.invoke(
    {
        "word": "tomato",
    }
)

'Explain about tomato. Use a jar of tomato paste. The tomato paste will absorb most from the tomato.\n\nTry and get out. Do not run from the street'